In [1]:
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys
sys.path.append("C:/Users/Daniel/OneDrive - VNU-HCMUS/Desktop/DA_Project/data_analytics_llm")
from src.models.llms import load_llm

MODEL_NAME = "gemini-1.5-flash"

llm = load_llm(MODEL_NAME)

llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_output_tokens=1000, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001AA457FAEC0>, default_metadata=())

In [3]:
from langchain.agents.agent_types import AgentType

In [4]:
df = pd.read_csv('../data/Sales Data.csv')

agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    allow_dangerous_code=True,
    verbose=True,
    return_intermediate_steps=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'df_head': '|    |   Unnamed: 0 |   Order ID | Product              |   Quantity Ordered |   Price Each | Order Date          | Purchase Address                       |   Month |   Sales | City          |   Hour |\n|---:|-------------:|-----------:|:---------------------|-------------------:|-------------:|:--------------------|:---------------------------------------|--------:|--------:|:--------------|-------:|\n|  0 |            0 |     295665 | Macbook Pro Laptop   |                  1 |      1700    | 2019-12-30 00:01:00 | 136 Church St, New York City, NY 10001 |      12 | 1700    | New York City |      0 |\n|  1 |            1 |     295666 | LG Washing Machine   |                  1 |       600    | 2019-12-29 07

In [5]:
query = "Describe the data"
response = agent(query)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_11452\246414373.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent(query)




> Entering new AgentExecutor chain...
Thought: I need to examine the dataframe to understand its structure and content.  The head() output shows the first five rows.  I should look at the data types of each column and the number of rows and columns.

Action: python_repl_ast
Action Input: 
```python
print(df.info())
print(df.describe())
```<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185950 entries, 0 to 185949
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        185950 non-null  int64  
 1   Order ID          185950 non-null  int64  
 2   Product           185950 non-null  object 
 3   Quantity Ordered  185950 non-null  int64  
 4   Price Each        185950 non-null  float64
 5   Order Date        185950 non-null  object 
 6   Purchase Address  185950 non-null  object 
 7   Month             185950 non-null  int64  
 8   Sales             185950 non-null  float64
 9   City         

In [34]:
query = "describe the data by using python code"
response = agent(query)



> Entering new AgentExecutor chain...
Thought: I need to use pandas' `describe()` method to get descriptive statistics of the numerical columns in the dataframe.

Action: python_repl_ast
Action Input: 
```python
print(df.describe())
```          Unnamed: 0       Order ID  Quantity Ordered     Price Each  \
count  185950.000000  185950.000000     185950.000000  185950.000000   
mean     8340.388475  230417.569379          1.124383     184.399735   
std      5450.554093   51512.737110          0.442793     332.731330   
min         0.000000  141234.000000          1.000000       2.990000   
25%      3894.000000  185831.250000          1.000000      11.950000   
50%      7786.000000  230367.500000          1.000000      14.950000   
75%     11872.000000  275035.750000          1.000000     150.000000   
max     25116.000000  319670.000000          9.000000    1700.000000   

               Month          Sales           Hour  
count  185950.000000  185950.000000  185950.000000  
mean   

In [37]:
for step in response["intermediate_steps"]:
    print(step[1], "\n")

          Unnamed: 0       Order ID  Quantity Ordered     Price Each  \
count  185950.000000  185950.000000     185950.000000  185950.000000   
mean     8340.388475  230417.569379          1.124383     184.399735   
std      5450.554093   51512.737110          0.442793     332.731330   
min         0.000000  141234.000000          1.000000       2.990000   
25%      3894.000000  185831.250000          1.000000      11.950000   
50%      7786.000000  230367.500000          1.000000      14.950000   
75%     11872.000000  275035.750000          1.000000     150.000000   
max     25116.000000  319670.000000          9.000000    1700.000000   

               Month          Sales           Hour  
count  185950.000000  185950.000000  185950.000000  
mean        7.059140     185.490917      14.413305  
std         3.502996     332.919771       5.423416  
min         1.000000       2.990000       0.000000  
25%         4.000000      11.950000      11.000000  
50%         7.000000      14.9500

In [10]:
print(response["intermediate_steps"][0][0].tool_input.strip("```python").strip("```"))


print(df.describe())



In [39]:
#
if "intermediate_steps" in response:
    for step in response["intermediate_steps"]:
        if step[0].tool == "python_repl_ast":  # Chỉ lấy code từ tool Python
            code = step[0].tool_input.strip("```python").strip("```")  # Lấy nội dung action_input và loại bỏ các ký tự không hợp lệ
            print("Extracted Python Code:\n", code)  # In ra để kiểm tra

            # Thực thi mã Python
            exec(code)

Extracted Python Code:
 
print(df.describe())

          Unnamed: 0       Order ID  Quantity Ordered     Price Each  \
count  185950.000000  185950.000000     185950.000000  185950.000000   
mean     8340.388475  230417.569379          1.124383     184.399735   
std      5450.554093   51512.737110          0.442793     332.731330   
min         0.000000  141234.000000          1.000000       2.990000   
25%      3894.000000  185831.250000          1.000000      11.950000   
50%      7786.000000  230367.500000          1.000000      14.950000   
75%     11872.000000  275035.750000          1.000000     150.000000   
max     25116.000000  319670.000000          9.000000    1700.000000   

               Month          Sales           Hour  
count  185950.000000  185950.000000  185950.000000  
mean        7.059140     185.490917      14.413305  
std         3.502996     332.919771       5.423416  
min         1.000000       2.990000       0.000000  
25%         4.000000      11.950000    

In [42]:
#
if "intermediate_steps" in response:
    for step in response["intermediate_steps"]:
        if step[0].tool == "python_repl_ast":  # Chỉ lấy code từ tool Python
            code = step[0].tool_input.strip("```python").strip("```")  # Lấy nội dung action_input và loại bỏ các ký tự không hợp lệ
            print("Extracted Python Code:\n", code)  # In ra để kiểm tra

            # Thực thi mã Python
            exec(code)

Extracted Python Code:
 
print(df.describe())

          Unnamed: 0       Order ID  Quantity Ordered     Price Each  \
count  185950.000000  185950.000000     185950.000000  185950.000000   
mean     8340.388475  230417.569379          1.124383     184.399735   
std      5450.554093   51512.737110          0.442793     332.731330   
min         0.000000  141234.000000          1.000000       2.990000   
25%      3894.000000  185831.250000          1.000000      11.950000   
50%      7786.000000  230367.500000          1.000000      14.950000   
75%     11872.000000  275035.750000          1.000000     150.000000   
max     25116.000000  319670.000000          9.000000    1700.000000   

               Month          Sales           Hour  
count  185950.000000  185950.000000  185950.000000  
mean        7.059140     185.490917      14.413305  
std         3.502996     332.919771       5.423416  
min         1.000000       2.990000       0.000000  
25%         4.000000      11.950000    